# Data reader-writer (loads)

This notebook is developed to read FES-2022 hydrogen consumption and split it by hourly fluctuation and clusters share (refer capacity-splitter).

This notbeook covers hydrogen loads from the following technologies: 
1. Industrial: including hourly fluctuations (historic)
2. Residential & Commercial: including hourly fluctuations (historic)
3. Transport (road): linear hourly consumption
4. Transport (rail, aviation, shipping - assumed as synthetic fuels): linear hourly consumption
5. Direct Air Carbon Capture and Storage: linear hourly consumption
6. Power Generation: fluctuations are extracted from PyPSA-GB and pro-rated to FES capacity.


In [1]:
import os
import sys
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)
os.getcwd()

'C:\\Users\\tatya\\OneDrive - University of Edinburgh\\01 Dissertation\\01 PyPSA\\PyPSA-GB-H2\\notebooks'

In [2]:
import pandas as pd
import numpy as np

In [3]:
# select scenario:
# scenario = 'Consumer Transformation'
# scenario = 'Falling Short'
# scenario = 'Leading the Way'
scenario = 'System Transformation'

In [4]:
# select year:
year = 2050

In [5]:
# select year (to write snapshots):
df = pd.DataFrame()
df['name'] = pd.date_range('2050-01-01 00:00:00','2050-12-31 23:00:00', freq='H')
df['weightings'] = 1
df.to_csv('..\data\LOPF_data\snapshots.csv', index=False)

In [6]:
# read FES-2022:
xls = pd.ExcelFile('..\data\FES2022\FES2022 Workbook V6.xlsx')
df1 = pd.read_excel(xls, 'WS1', header=7)
# get column A name - 'Fuel Type'
col_1 = df1.columns[0]
# get column B name - 'Scenario'
col_2 = df1.columns[1]
# get column D name - 'Category'
col_3 = df1.columns[3]
# get column E name - 'Sector'
col_4 = df1.columns[4]

### Get yearly consumption per Sector from FES

In [7]:
# get partial dataframe with rows filtered by value in column A & B & D & E - by Scenario & year

# Industrial Sector:
df_industry = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Demand') & (df1[col_4].isin(['Industrial']))]
df_industry = df_industry.filter(['Sector', year], axis=1)
df_industry = df_industry.set_index(['Sector'])

# Residential & Commercial Sectors (heating):
df_heating = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Demand') & (df1[col_4].isin(['Residential', 'Commercial']))]
df_heating = df_heating.filter(['Sector', year], axis=1)
df_heating = df_heating.set_index(['Sector'])

# Transport Sector (road):
df_road = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Demand') & (df1[col_4].isin(['Road Transport']))]
df_road = df_road.filter(['Sector', year], axis=1)
df_road = df_road.set_index(['Sector'])

# Transport Sector (shipping, aviation, rail):
df_shipping = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Demand') & (df1[col_4].isin(['Shipping','Aviation','Rail']))]
df_shipping = df_shipping.filter(['Sector', year], axis=1)
df_shipping = df_shipping.set_index(['Sector'])
# sum shipping + aviation + rail
df_shipping.loc['shipping',:] = df_shipping.sum(axis=0, numeric_only=True)
df_shipping = df_shipping.filter(['shipping'], axis=0)

# Direct Air Carbon Capture and Storage
df_daccs = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]== scenario) & (df1[col_3]=='Demand') & (df1[col_4].isin(['Direct Air Carbon Capture and Storage']))]
df_daccs = df_daccs.filter(['Sector', year], axis=1)
df_daccs = df_daccs.set_index(['Sector'])

# Power
df_power = df1.loc[(df1[col_1]=='Hydrogen') & (df1[col_2]==scenario) & (df1[col_3]=='Demand') & (df1[col_4]=='Power Generation')]
df_power = df_power.filter(['Sector', year], axis=1)
df_power = df_power.set_index(['Sector'])

In [8]:
# read capacity-splitter by clusters:
xls = pd.ExcelFile('..\data\capacity-splitter\capacity-splitter.xlsx')
df2 = pd.read_excel(xls, 'loads-share', header=0)

### 1. Industial

In [9]:
# read historical data of industry consumption 'industrial-historic-output':
df3 = pd.read_csv('..\data\loads\historic-industrial\industrial-historic-output.csv')

In [10]:
# get hourly consumption (& convert TWh to MWh):
df3['industrial'] = df3['industrial, share'].mul(df_industry.iloc[0][year]).round(10) * 1e6
df3 = df3.filter(['industrial'], axis=1)
df3['name'] = df['name']

In [11]:
# split values by Industrial Clusters based on assumed % (check share = 1):

df3['St Fergus'] = df3['industrial'].mul(df2.iloc[0]['St Fergus'])
df3['Grangemouth'] = df3['industrial'].mul(df2.iloc[0]['Grangemouth'])
df3['Teesside'] = df3['industrial'].mul(df2.iloc[0]['Teesside'])
df3['Humberside'] = df3['industrial'].mul(df2.iloc[0]['Humberside'])
df3['Theddlethorpe'] = df3['industrial'].mul(df2.iloc[0]['Theddlethorpe'])
df3['Merseyside'] = df3['industrial'].mul(df2.iloc[0]['Merseyside'])
df3['Barrow'] = df3['industrial'].mul(df2.iloc[0]['Barrow'])
df3['Bacton'] = df3['industrial'].mul(df2.iloc[0]['Bacton'])
df3['Grain LNG'] = df3['industrial'].mul(df2.iloc[0]['Grain LNG'])
df3['Southampton'] = df3['industrial'].mul(df2.iloc[0]['Southampton'])
df3['South Wales'] = df3['industrial'].mul(df2.iloc[0]['South Wales'])

df3 = df3.drop(['industrial'], axis=1)
df3 = df3.set_index(['name'])

In [12]:
# check total hydrogen consumption per sector / year:
df3.values.sum() / 1e6

87.733101252

In [13]:
df3.to_csv('..\data\loads\working\h2-industrial.csv', index=True)

### 2. Heating

In [14]:
# read historical data of heating 'heating-historic-output':
df4 = pd.read_csv('..\data\loads\historic-heating\heating-historic-output.csv')

In [15]:
# get hourly consumption (& convert TWh to MWh):
df4['residential'] = df4['residential, share'].mul(df_heating.iloc[0][year]).round(10) * 1e6
df4['commercial'] = df4['commercial, share'].mul(df_heating.iloc[1][year]).round(10) * 1e6
# sum 'residential' and 'commercial' as assumed the same share between clusters:
df4['heating'] = df4['residential'] + df4['commercial']
df4 = df4.filter(['heating'], axis=1)
df4['name'] = df['name']

In [16]:
# split values by Industrial Clusters based on assumed % (check share = 1):

df4['St Fergus'] = df4['heating'].mul(df2.iloc[1]['St Fergus'])
df4['Grangemouth'] = df4['heating'].mul(df2.iloc[1]['Grangemouth'])
df4['Teesside'] = df4['heating'].mul(df2.iloc[1]['Teesside'])
df4['Humberside'] = df4['heating'].mul(df2.iloc[1]['Humberside'])
df4['Theddlethorpe'] = df4['heating'].mul(df2.iloc[1]['Theddlethorpe'])
df4['Merseyside'] = df4['heating'].mul(df2.iloc[1]['Merseyside'])
df4['Barrow'] = df4['heating'].mul(df2.iloc[1]['Barrow'])
df4['Bacton'] = df4['heating'].mul(df2.iloc[1]['Bacton'])
df4['Grain LNG'] = df4['heating'].mul(df2.iloc[1]['Grain LNG'])
df4['Southampton'] = df4['heating'].mul(df2.iloc[1]['Southampton'])
df4['South Wales'] = df4['heating'].mul(df2.iloc[1]['South Wales'])

df4 = df4.drop(['heating'], axis=1)
df4 = df4.set_index(['name'])

In [17]:
# check total hydrogen consumption per sector / year:
df4.values.sum() / 1e6

191.29726013689995

In [18]:
df4.to_csv('..\data\loads\working\h2-heating.csv', index=True)

### 3. Transport (Road)

In [19]:
# read snapshots and add daily equal-share:
df5 = pd.read_csv('..\data\LOPF_data\snapshots.csv')
df5['equal share'] = 1 / 8760

In [20]:
# get hourly consumption (& convert TWh to MWh):
df5['road'] = df5['equal share'].mul(df_road.iloc[0][year]).round(10) * 1e6
df5 = df5.filter(['road'], axis=1)
df5['name'] = df['name']

In [21]:
# split values by Industrial Clusters based on assumed % (check share = 1):

df5['St Fergus'] = df5['road'].mul(df2.iloc[3]['St Fergus'])
df5['Grangemouth'] = df5['road'].mul(df2.iloc[3]['Grangemouth'])
df5['Teesside'] = df5['road'].mul(df2.iloc[3]['Teesside'])
df5['Humberside'] = df5['road'].mul(df2.iloc[3]['Humberside'])
df5['Theddlethorpe'] = df5['road'].mul(df2.iloc[3]['Theddlethorpe'])
df5['Merseyside'] = df5['road'].mul(df2.iloc[3]['Merseyside'])
df5['Barrow'] = df5['road'].mul(df2.iloc[3]['Barrow'])
df5['Bacton'] = df5['road'].mul(df2.iloc[3]['Bacton'])
df5['Grain LNG'] = df5['road'].mul(df2.iloc[3]['Grain LNG'])
df5['Southampton'] = df5['road'].mul(df2.iloc[3]['Southampton'])
df5['South Wales'] = df5['road'].mul(df2.iloc[3]['South Wales'])

df5 = df5.drop(['road'], axis=1)
df5 = df5.set_index(['name'])

In [22]:
# check total hydrogen consumption per sector / year:
df5.values.sum() / 1e6

56.224379832000004

In [23]:
df5.to_csv('..\data\loads\working\h2-transport.csv', index=True)

### 4. Transport (Synthetic Fuels)

In [24]:
# read snapshots and add daily equal-share:
df10 = pd.read_csv('..\data\LOPF_data\snapshots.csv')
df10['equal share'] = 1 / 8760

In [25]:
# get hourly consumption (& convert TWh to MWh):
df10['shipping'] = df10['equal share'].mul(df_shipping.iloc[0][year]).round(10) * 1e6
df10 = df10.filter(['shipping'], axis=1)
df10['name'] = df['name']

In [26]:
# split values by Industrial Clusters based on assumed % (check share = 1):

df10['St Fergus'] = df10['shipping'].mul(df2.iloc[4]['St Fergus'])
df10['Grangemouth'] = df10['shipping'].mul(df2.iloc[4]['Grangemouth'])
df10['Teesside'] = df10['shipping'].mul(df2.iloc[4]['Teesside'])
df10['Humberside'] = df10['shipping'].mul(df2.iloc[4]['Humberside'])
df10['Theddlethorpe'] = df10['shipping'].mul(df2.iloc[4]['Theddlethorpe'])
df10['Merseyside'] = df10['shipping'].mul(df2.iloc[4]['Merseyside'])
df10['Barrow'] = df10['shipping'].mul(df2.iloc[4]['Barrow'])
df10['Bacton'] = df10['shipping'].mul(df2.iloc[4]['Bacton'])
df10['Grain LNG'] = df10['shipping'].mul(df2.iloc[4]['Grain LNG'])
df10['Southampton'] = df10['shipping'].mul(df2.iloc[4]['Southampton'])
df10['South Wales'] = df10['shipping'].mul(df2.iloc[4]['South Wales'])

df10 = df10.drop(['shipping'], axis=1)
df10 = df10.set_index(['name'])

In [27]:
# check total hydrogen consumption per sector / year:
df10.values.sum() / 1e6

81.85887207599997

In [28]:
df10.to_csv('..\data\loads\working\h2-synthetic-fuels.csv', index=True)

### 5. Direct Air Carbon Capture and Storage

In [29]:
# read snapshots and add daily equal-share:
df6 = pd.read_csv('..\data\LOPF_data\snapshots.csv')
df6['equal share'] = 1 / 8760

In [30]:
# split FES annual to hourly demand:
df6['daccs'] = df6['equal share'].mul(df_daccs.iloc[0][year]).round(10) * 1e6
df6 = df6.filter(['daccs'], axis=1)
df6['name'] = df['name']

In [31]:
# split values by Industrial Clusters based on assumed % (check share = 1):

df6['St Fergus'] = df6['daccs'].mul(df2.iloc[7]['St Fergus'])
df6['Grangemouth'] = df6['daccs'].mul(df2.iloc[7]['Grangemouth'])
df6['Teesside'] = df6['daccs'].mul(df2.iloc[7]['Teesside'])
df6['Humberside'] = df6['daccs'].mul(df2.iloc[7]['Humberside'])
df6['Theddlethorpe'] = df6['daccs'].mul(df2.iloc[7]['Theddlethorpe'])
df6['Merseyside'] = df6['daccs'].mul(df2.iloc[7]['Merseyside'])
df6['Barrow'] = df6['daccs'].mul(df2.iloc[7]['Barrow'])
df6['Bacton'] = df6['daccs'].mul(df2.iloc[7]['Bacton'])
df6['Grain LNG'] = df6['daccs'].mul(df2.iloc[7]['Grain LNG'])
df6['Southampton'] = df6['daccs'].mul(df2.iloc[7]['Southampton'])
df6['South Wales'] = df6['daccs'].mul(df2.iloc[7]['South Wales'])

df6 = df6.drop(['daccs'], axis=1)
df6 = df6.set_index(['name'])

In [32]:
# check total hydrogen consumption per sector / year:
df6.values.sum() / 1e6

0.0

In [33]:
df6.to_csv('..\data\loads\working\h2-daccs.csv', index=True)

### 6. Power

In [34]:
# scenario is selected based on earlier input: 
if scenario == 'Consumer Transformation': 
    path = '..\data\PyPSA-GB\h2-power-loads\CT\year-2050'
elif scenario == 'Falling Short': 
    path = '..\data\PyPSA-GB\h2-power-loads\FS\year-2050'
elif scenario == 'Leading the Way':
    path = '..\data\PyPSA-GB\h2-power-loads\LW\year-2050'
elif scenario == 'System Transformation':
    path = '..\data\PyPSA-GB\h2-power-loads\ST\year-2050'

In [35]:
path

'..\\data\\PyPSA-GB\\h2-power-loads\\ST\\year-2050'

In [36]:
# read and combine data from PyPSA-GB (run done in 2 x steps due to processing limitation):
df_1 = pd.read_csv(path+'\Hydrogen-1st-half.csv')
df_2 = pd.read_csv(path+'\Hydrogen-2nd-half.csv')
df7 = pd.concat([df_1, df_2], ignore_index=True, sort=False)
df7['name'] = df7['snapshot']
df7 = df7.set_index(['name'])
df7 = df7.loc[:, df7.columns.str.contains('Hydrogen')]

In [37]:
# split Hydrogen Power Plants by Clusters based on location (manual - low accuracy):

df7['St Fergus'] = df7['Peterhead Hydrogen']
df7['Grangemouth'] = 0
df7['Teesside'] = 0
df7['Humberside'] = df7['Castleford Hydrogen'] + df7['South Humber Bank Hydrogen'] + df7['Keadby Hydrogen'] + df7['Saltend* Hydrogen'] + df7['VPI Immingham* Hydrogen']
df7['Theddlethorpe'] = df7['Staythorpe C Hydrogen'] + df7['Cottam Development Centre Hydrogen']
df7['Merseyside'] = df7['Blackburn Hydrogen'] + df7['Sandbach Hydrogen'] + df7['Thornhill Hydrogen'] + df7['West Burton CCGT Hydrogen'] + df7['Carrington Hydrogen'] + df7['Fellside CHP* Hydrogen'] + df7['Rocksavage Hydrogen'] + df7['Connahs Quay Hydrogen']
df7['Bacton'] = df7['Sutton Bridge Hydrogen'] + df7['Corby Hydrogen'] + df7['Spalding Hydrogen'] + df7['Great Yarmouth Hydrogen'] + df7['Little Barford Hydrogen']
df7['Grain LNG'] = df7['Damhead Creek Hydrogen'] + df7['Rye House Hydrogen'] + df7['Medway Hydrogen'] + df7['Enfield Hydrogen'] + df7['Grain CHP* Hydrogen']
df7['Southampton'] = df7['Shoreham Hydrogen'] + df7['Langage Hydrogen'] + df7['Marchwood Hydrogen'] + df7['Didcot B Hydrogen']
df7['South Wales'] = df7['Baglan Bay Hydrogen'] + df7['Severn Power Hydrogen'] + df7['Coryton Hydrogen'] + df7['Pembroke Hydrogen'] + df7['Seabank Hydrogen']

df7 = df7.filter(['St Fergus','Grangemouth','Teesside','Humberside','Theddlethorpe','Merseyside','Bacton','Grain LNG','Southampton','South Wales'], axis=1)

df7.values.sum() / 1e6

12.75053358258836

In [38]:
# adjust to total Hydrogen consumption of PyPSA-GB output to FES:
percent = df7.values.sum() / 1e6 / df_power.values.sum()
df7 = df7.div(percent)
# check total hydrogen consumption per sector / year:
df7.values.sum() / 1e6

12.600000000000001

In [39]:
df7.to_csv('..\data\loads\working\h2-power.csv', index=True)

### Merge all clusters in one table and re-arrange data for PyPSA

In [40]:
import glob

In [41]:
path = '..\data\loads\working'

In [42]:
all_files = glob.glob(os.path.join(path, "*.csv"))
df8 = pd.concat((pd.read_csv(f) for f in all_files), axis=1, ignore_index=False)
df8 = df8.drop(['name'], axis=1)
df8 = df8.groupby(level=0,axis=1).sum()
df8['name'] = df['name']
df8 = df8.set_index(['name'])

In [43]:
# check total hydrogen consumption per scenario / year:
df8.values.sum() / 1e6

429.71361329689995

In [44]:
df8.to_csv('..\data\LOPF_data\loads-p_set.csv', index=True)

### Write Loads csv

In [45]:
df9 = pd.read_csv('..\data\loads\loads-names.csv')
df9['bus'] = df9['name']

In [46]:
df9.to_csv('..\data\LOPF_data\loads.csv', index=False)